In [1]:
def read_config_values():
    """Reads values from the configuration files and returns dictionary"""
    
    if config_file_exists():
        import image_caption_config
    else:
        raise ValueError ('Missing file image_caption_config.py, please refer to documentation') 
    
    # Values in a hidden config file override the config file, useful to prevent user config settings being pushed to github
    if hidden_config_file_exists():
        import _image_caption_config
        if image_caption_config.config.keys() != _image_caption_config.config.keys():
            raise ValueError ('keys in config files dont match hidden config file') 
        else:
            config_values = _image_caption_config.config
    else:
        config_values = image_caption_config.config
    
    return (config_values)


def config_file_exists():
    """ Check if a configuration file exists"""
    return(os.path.isfile('image_caption_config.py'))


def hidden_config_file_exists():
    """ Check if hidden configuration file exists"""
    return(os.path.isfile('_image_caption_config.py'))


def in_dir_exists(in_dir):
    """Check if in_dir directory exists"""
    return(os.path.isdir(in_dir))
            
            
def out_dir_exists(out_dir):
    """Check if out_dir directory exists"""
    return(os.path.isdir(out_dir))


def create_out_dir(out_dir):
    """Create out_dir"""
    pathlib.Path(out_dir).mkdir(parents=True)
        

class CaptionedPhoto:
    
    def __init__(self, file_path, img_ratios):
        """Opens image, sets the frame and border ratios and transpose if required"""
        self.original_image = Image.open(file_path)
        self.frame_ratio, self.lower_border_ratio, self.other_borders_ratio = img_ratios
        
        # If orientation is stored via exif metadata then transpose image and remove the orientation metadata
        self.original_image = ImageOps.exif_transpose(self.original_image)

        
    def _max_side_length(self):
        """returns the max of height and width of the image"""
        img_width, img_height = self.original_image.size
        return(max(img_width, img_height))

    
    def _original_image_width(self):
        """Returns the original image width"""
        return self.original_image.size[0]

    
    def _original_image_height(self):
        """Returns the original image height"""
        return self.original_image.size[1]

        
    def frame_width(self):
        """Returns frame width"""
        return(int(self._max_side_length() * self.frame_ratio))

    
    def lower_border_width(self):
        """Returns lower border width"""
        return(int(self._max_side_length() * self.lower_border_ratio))

    
    def other_borders_width(self):
        """Returns widths of borders other than lower border"""
        return(int(self._max_side_length() * self.other_borders_ratio))

    
    def _border_image(self):
        """Returns rectangle reprepsenting border image excluding photo and frame"""
        border_image_width = self._original_image_width() + self.other_borders_width() * 2
        border_image_height = self._original_image_height() + self.lower_border_width() + self.other_borders_width()
        img_border = Image.new('RGB', (border_image_width, border_image_height), 'RGB(220,220,220)')
        return(img_border)
    
    
    def _border_image_width(self):
        """Returns the border image width"""
        return self._border_image().size[0]

    
    def _border_image_height(self):
        """Returns the border image height"""
        return self._border_image().size[1]
    
    
    def _frame_image(self):
        """Returns rectangle representing the frame image excluding border and photo"""
        frame_img_width = self._border_image_width() + self.frame_width()*2
        frame_img_height = self._border_image_height() + self.frame_width()*2
        img_frame = Image.new('RGB', (frame_img_width, frame_img_height), 'RGB(120,75,50)')
        return(img_frame)
        
        
    def _output_image(self):
        """Returns original image inside border, indside frame"""
        img_a = self._border_image()
        img_a.paste(self.original_image, (self.other_borders_width(), self.other_borders_width()))
        img_b = self._frame_image()
        img_b.paste(img_a, (self.frame_width(), self.frame_width()))
        return (img_b)
        
        
    def save(self, path):
        """Saves the captioned image to path directory"""
        self._output_image().save(path)
         
if __name__ == "__main__":
    
    from PIL import Image, ImageDraw, ImageFont, ImageOps
    import os
    import pathlib
  
    # Read configuration inputs into a dictionary
    inputs = read_config_values()
    
#     run_directory_error_checks(in_dir, out_dir)
#     create_out_dir_if_it_doesnt_exist(out_dir)
    
#     for filename_and_extension in os.listdir(in_dir):
#         image_extensions_lower_case = ['.jpeg', '.jpg', '.png']
#         base_filename, file_extension = os.path.splitext(filename_and_extension)        
#         if file_extension.lower() in image_extensions_lower_case:
#             input_filepath = os.path.join(in_dir, base_filename + file_extension)
#             print ('processing ' + input_filepath)
#             output_filepath = os.path.join(out_dir, base_filename + file_extension)
#             my_photo = CaptionedPhoto(input_filepath, image_ratios)
#             my_photo.save(output_filepath)
#     print('\nProcessing complete...')

In [2]:
inputs['in_dir']

'/home/charl/OneDrive/Documents_Charl/Computer_Technical/Programming_GitHub/AddImageCaption/Test_In_Folder/'